In [81]:
%pylab inline
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import scipy.optimize as opt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import log_loss
import itertools

Populating the interactive namespace from numpy and matplotlib


In [4]:
df = pd.read_csv('ChurnData.csv')
df

,tenure,age,address,income,ed,employ,equip,callcard,wireless,longmon,...,pager,internet,callwait,confer,ebill,loglong,logtoll,lninc,custcat,churn
0,11.0,33.0,7.0,136.0,5.0,5.0,0.0,1.0,1.0,4.40,...,1.0,0.0,1.0,1.0,0.0,1.482,3.033,4.913,4.0,1.0
1,33.0,33.0,12.0,33.0,2.0,0.0,0.0,0.0,0.0,9.45,...,0.0,0.0,0.0,0.0,0.0,2.246,3.240,3.497,1.0,1.0
2,23.0,30.0,9.0,30.0,1.0,2.0,0.0,0.0,0.0,6.30,...,0.0,0.0,0.0,1.0,0.0,1.841,3.240,3.401,3.0,0.0
3,38.0,35.0,5.0,76.0,2.0,10.0,1.0,1.0,1.0,6.05,...,1.0,1.0,1.0,1.0,1.0,1.800,3.807,4.331,4.0,0.0
4,7.0,35.0,14.0,80.0,2.0,15.0,0.0,1.0,0.0,7.10,...,0.0,0.0,1.0,1.0,0.0,1.960,3.091,4.382,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,55.0,44.0,24.0,83.0,1.0,23.0,0.0,1.0,0.0,17.35,...,0.0,0.0,0.0,1.0,0.0,2.854,3.199,4.419,3.0,0.0
196,34.0,23.0,3.0,24.0,1.0,7.0,0.0,1.0,0.0,6.00,...,0.0,0.0,1.0,1.0,0.0,1.792,3.332,3.178,3.0,0.0
197,6.0,32.0,10.0,47.0,1.0,10.0,0.0,1.0,0.0,3.85,...,0.0,0.0,1.0,1.0,0.0,1.348,3.168,3.850,3.0,0.0
198,24.0,30.0,0.0,25.0,4.0,5.0,0.0,1.0,1.0,8.70,...,1.0,1.0,1.0,1.0,1.0,2.163,3.866,3.219,4.0,1.0


In [6]:
df = df[['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip',   'callcard', 'wireless','churn']]

In [7]:
df

,tenure,age,address,income,ed,employ,equip,callcard,wireless,churn
0,11.0,33.0,7.0,136.0,5.0,5.0,0.0,1.0,1.0,1.0
1,33.0,33.0,12.0,33.0,2.0,0.0,0.0,0.0,0.0,1.0
2,23.0,30.0,9.0,30.0,1.0,2.0,0.0,0.0,0.0,0.0
3,38.0,35.0,5.0,76.0,2.0,10.0,1.0,1.0,1.0,0.0
4,7.0,35.0,14.0,80.0,2.0,15.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
195,55.0,44.0,24.0,83.0,1.0,23.0,0.0,1.0,0.0,0.0
196,34.0,23.0,3.0,24.0,1.0,7.0,0.0,1.0,0.0,0.0
197,6.0,32.0,10.0,47.0,1.0,10.0,0.0,1.0,0.0,0.0
198,24.0,30.0,0.0,25.0,4.0,5.0,0.0,1.0,1.0,1.0


In [11]:
df['churn'] = df['churn'].astype('int')

<ipython-input-11-937fcf4ed1e1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['churn'] = df['churn'].astype('int')


In [12]:
df['churn']

0      1
1      1
2      0
3      0
4      0
      ..
195    0
196    0
197    0
198    1
199    0
Name: churn, Length: 200, dtype: int32

In [57]:
y = np.asarray(df['churn'])

In [58]:
y

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0])

In [59]:
x = np.asarray(df[['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip']])
x

array([[11., 33.,  7., ...,  5.,  5.,  0.],
       [33., 33., 12., ...,  2.,  0.,  0.],
       [23., 30.,  9., ...,  1.,  2.,  0.],
       ...,
       [ 6., 32., 10., ...,  1., 10.,  0.],
       [24., 30.,  0., ...,  4.,  5.,  0.],
       [61., 50., 16., ...,  2., 22.,  1.]])

In [60]:
x = preprocessing.StandardScaler().fit(x).transform(x)

In [61]:
x[0:20]

array([[-1.13518441, -0.62595491, -0.4588971 ,  0.4751423 ,  1.6961288 ,
        -0.58477841, -0.85972695],
       [-0.11604313, -0.62595491,  0.03454064, -0.32886061, -0.6433592 ,
        -1.14437497, -0.85972695],
       [-0.57928917, -0.85594447, -0.261522  , -0.35227817, -1.42318853,
        -0.92053635, -0.85972695],
       [ 0.11557989, -0.47262854, -0.65627219,  0.00679109, -0.6433592 ,
        -0.02518185,  1.16316   ],
       [-1.32048283, -0.47262854,  0.23191574,  0.03801451, -0.6433592 ,
         0.53441472, -0.85972695],
       [ 1.505318  ,  0.83064562,  0.52797838,  0.35024865, -1.42318853,
         1.54168853, -0.85972695],
       [ 0.3008783 , -0.08931261, -0.4588971 , -0.29763719, -0.6433592 ,
        -0.24902047,  1.16316   ],
       [-1.22783362, -1.54591314, -1.05102238, -0.45375426, -0.6433592 ,
        -0.92053635, -0.85972695],
       [-0.02339392,  0.67731925,  1.41616631,  0.50636572, -0.6433592 ,
         1.2059306 , -0.85972695],
       [ 0.62515053,  0.7539

In [62]:
x_train  , x_test , y_train , y_test = train_test_split(x , y , test_size = 0.2 , random_state = 4)

In [63]:
print(x_train)

[[ 1.08839657  0.14067695 -0.75495974 ... -0.6433592   1.31784991
  -0.85972695]
 [ 0.85677355  0.06401376 -1.14970993 ...  0.91629947 -0.92053635
   1.16316   ]
 [ 0.94942275  0.9073088   1.12010367 ...  1.6961288   0.42249541
   1.16316   ]
 ...
 [ 0.71779974  0.60065606  1.71222895 ... -1.42318853  0.64633403
  -0.85972695]
 [-1.50578124 -0.7026181  -0.75495974 ... -0.6433592   0.08673747
   1.16316   ]
 [-0.53296457 -0.47262854 -0.16283445 ...  1.6961288  -0.4728591
   1.16316   ]]


In [64]:
print(x_test[0:10])

[[ 0.53250132 -0.08931261  0.42929083  0.40488962  0.91629947  0.19865678
   1.16316   ]
 [ 0.81044894 -0.62595491 -1.05102238 -0.11810256 -1.42318853 -0.4728591
  -0.85972695]
 [-0.25501694 -1.00927084 -1.05102238 -0.4303367  -1.42318853 -0.24902047
  -0.85972695]
 [ 0.76412434 -0.1659758  -0.55758464  0.34244279  0.13647013  0.87017266
  -0.85972695]
 [-0.11604313 -0.62595491  0.03454064 -0.32886061 -0.6433592  -1.14437497
  -0.85972695]
 [-1.41313204  0.44732969 -0.4588971  -0.22738451 -1.42318853 -0.4728591
  -0.85972695]
 [-0.48663996 -0.93260766 -0.261522   -0.15713183  0.91629947 -1.03245566
  -0.85972695]
 [ 0.34720291 -0.93260766 -0.75495974 -0.32886061 -1.42318853  0.31057609
  -0.85972695]
 [-1.41313204 -0.7026181  -0.55758464 -0.32886061  0.91629947 -0.80861704
   1.16316   ]
 [ 1.04207196 -0.39596535  0.13322819 -0.28202549 -0.6433592  -0.24902047
  -0.85972695]]


In [65]:
LR = LogisticRegression(C=0.01, solver='liblinear').fit(x_train,y_train)

In [66]:
y_pred = LR.predict(x_test)

In [67]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0])

In [68]:
y_test

array([0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0])

In [80]:
print (classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.96      0.83        25
           1       0.86      0.40      0.55        15

    accuracy                           0.75        40
   macro avg       0.79      0.68      0.69        40
weighted avg       0.78      0.75      0.72        40



In [83]:
yhat_prob = LR.predict_proba(x_test)
log_loss(y_test, yhat_prob)

0.6017092478101185